### Generate flight policy data



In [ ]:
from openai import AzureOpenAI
from datetime import datetime  
import os
from pathlib import Path  
import json
import random
import uuid
from tenacity import retry, wait_random_exponential, stop_after_attempt  
import pandas as pd
from dotenv import load_dotenv
env_path = Path('.') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings

env_path = Path('.') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
chat_engine =os.getenv("AZURE_OPENAI_EVALUATOR_DEPLOYMENT")
emb_engine = os.getenv("AZURE_OPENAI_EMB_DEPLOYMENT")

client = AzureOpenAI(
  api_key=os.environ.get("AZURE_OPENAI_API_KEY"),  
  api_version="2023-12-01-preview",
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
)


In [ ]:
def get_embedding(text, model=emb_engine):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_hotel_policy():
    messages = [{"role":"system", "content":"You are a helpful assistant designed to output JSON."}, {"role":"user", "content":'generate an example of hotel policy in JSON format {{"policy_name": name of policy, "policy text":"detail of policy with at least 150 words"}}"'}]
    response = client.chat.completions.create(
        model=chat_engine, # The deployment name you chose when you deployed the GPT-35-turbo or GPT-4 model.
        messages=messages,
        response_format={ "type": "json_object" },
    max_tokens=300,

    )
    output= json.loads(response.choices[0].message.content)
    if len(output.keys())!=2 or "policy_name" not in output.keys() or "policy_text" not in output.keys():
        raise Exception("Policy generation failed")
    return output

hotels = [
    "Marriott International",
    "Hilton Worldwide",
    "Hyatt Hotels Corporation",
    "Wyndham Hotels & Resorts",
    "InterContinental Hotels Group",
    "Choice Hotels International",
    "AccorHotels",
    "Best Western International",
    "Radisson Hotel Group",
    "Melia Hotels International",
    "Four Seasons Hotels and Resorts",
    "Shangri-La Hotels and Resorts",
    "Mandarin Oriental Hotel Group",
    "Fairmont Hotels & Resorts",
    "Ritz-Carlton Hotel Company",
    "JW Marriott Hotels",
    "W Hotels",
    "St. Regis Hotels & Resorts",
    "Kimpton Hotels & Restaurants",
    "Loews Hotels & Co"
]
hotel_policy_list = []
for hotel in hotels:
    for i in range(10):
        hotel_policy = {}
        hotel_policy["id"] = str(uuid.uuid4())
        hotel_policy["hotel"] = hotel
        policy = generate_hotel_policy()
        hotel_policy["policy_name"] = policy["policy_name"]
        hotel_policy["policy_text"] = policy["policy_text"]
        hotel_policy["policy_text_embedding"] = get_embedding(policy["policy_text"])
        hotel_policy["policy_name_embedding"] = get_embedding(policy["policy_name"])
        hotel_policy_list.append(hotel_policy)



In [ ]:
#write holtel policy_list to json file
with open('data/hotel_policy.json', 'w') as f:
    json.dump(hotel_policy_list, f)

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_policy():
    messages = [{"role":"system", "content":"You are a helpful assistant designed to output JSON."}, {"role":"user", "content":'generate an example of airline policy in JSON format {{"policy_name": name of policy, "policy text":"detail of policy with at least 150 words"}}"'}]
    response = client.chat.completions.create(
        model=chat_engine, # The deployment name you chose when you deployed the GPT-35-turbo or GPT-4 model.
        messages=messages,
        response_format={ "type": "json_object" },
    max_tokens=300,

    )
    output= json.loads(response.choices[0].message.content)
    if len(output.keys())!=2 or "policy_name" not in output.keys() or "policy_text" not in output.keys():
        raise Exception("Policy generation failed")
    return output

us_airlines = [
    "American Airlines",
    "Delta Air Lines",
    "United Airlines",
    "Southwest Airlines",
    "JetBlue Airways",
    "Alaska Airlines",
    "Spirit Airlines",
    "Frontier Airlines",
    "Hawaiian Airlines",
    "Allegiant Air"
]
policy_list = []
for airline in us_airlines:
    for i in range(10):
        airline_policy = {}
        airline_policy["id"] = str(uuid.uuid4())
        airline_policy["airline"] = airline
        policy = generate_policy()
        airline_policy["policy_name"] = policy["policy_name"]
        airline_policy["policy_text"] = policy["policy_text"]
        airline_policy["policy_text_embedding"] = get_embedding(policy["policy_text"])
        airline_policy["policy_name_embedding"] = get_embedding(policy["policy_name"])
        policy_list.append(airline_policy)
    


        



In [ ]:
#write flight policy_list to json file
with open('data/flight_policy.json', 'w') as f:
    json.dump(policy_list, f)

In [ ]:
with open('data/flight_policy.json', 'r') as f:
    flight_policy_list = json.load(f)
for chunk_id,_, chunk_content,_,_, vector in flight_policy_list:  
    print(chunk_id, chunk_content, vector)
    break

In [ ]:
flight_policy_list

### Generate/reset flight & hotel data

In [ ]:
policy

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(String, primary_key=True)
    name = Column(String)
    flights = relationship('Flight', backref='customer')

class Flight(Base):
    __tablename__ = 'flights'
    id = Column(Integer, primary_key=True, autoincrement=True)
    customer_id = Column(String, ForeignKey('customers.id'))
    ticket_num = Column(String)
    flight_num = Column(String)
    airline = Column(String)
    seat_num = Column(String)
    departure_airport = Column(String)
    arrival_airport = Column(String)
    departure_time = Column(DateTime)
    arrival_time = Column(DateTime)
    ticket_class = Column(String)
    gate = Column(String)
    status = Column(String)

engine = create_engine('sqlite:///data/flight_db.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()


In [ ]:

# Insert initial customer data
customer = Customer(id="12345", name="John Doe")
session.add(customer)
session.commit()


In [ ]:

# Insert initial flight data for the customer
initial_flights = [
    Flight(
        customer_id="12345",
        ticket_num="t00113",
        flight_num="AB123",
        airline="American Airlines",
        seat_num="12A",
        departure_airport="Airport A",
        arrival_airport="Airport B",
        departure_time=datetime(2023, 7, 1, 15, 30),
        arrival_time=datetime(2023, 7, 1, 18, 45),
        ticket_class="Economy",
        gate="G5",
        status="open"
    ),
    # Add more flights if needed
]

session.add_all(initial_flights)  
session.commit()  
  
print("Initial flight data has been inserted for customer John Doe.")  

In [ ]:
def load_user_flight_info(user_id):
    # Load flight information from SQLite using SQLAlchemy
    user_id=user_id.strip()
    print("user_id", user_id)
    matched_flights = session.query(Flight).filter_by(customer_id=user_id, status="open").all()
    
    flights_info = []
    for flight in matched_flights:
        flight_info = {
            'airline': flight.airline,  # Assuming you have this field in the Flight model.
            'flight_num': flight.flight_num,
            'seat_num': flight.seat_num,
            'departure_airport': flight.departure_airport,
            'arrival_airport': flight.arrival_airport,
            'departure_time': flight.departure_time.strftime('%Y-%m-%d %H:%M'),
            'arrival_time': flight.arrival_time.strftime('%Y-%m-%d %H:%M'),
            'ticket_class': flight.ticket_class,
            'ticket_num': flight.ticket_num,
            'gate': flight.gate,
            'status': flight.status
        }
        flights_info.append(flight_info)

    if not flights_info:
        return "Sorry, we cannot find any flight information for you."

    return str(flights_info)

load_user_flight_info("12345")

In [ ]:
from datetime import datetime

# Original time data
time_data = '2023-06-30T22:00:00'

# Parse the original time data
parsed_time = datetime.strptime(time_data, '%Y-%m-%d %H:%M')

# Format it to the desired format
formatted_time = parsed_time.strftime('%Y-%m-%d %H:%M')

print(formatted_time)


### Reservation data creation

In [ ]:
from sqlalchemy import create_engine, Column, String, Integer, DateTime, ForeignKey  
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import relationship, sessionmaker  
from datetime import datetime  
  
Base = declarative_base()  

class Customer(Base):  
    __tablename__ = 'customers'  
    id = Column(String, primary_key=True)  
    name = Column(String)  
    reservations = relationship('Reservation', backref='customer')  
  
class Reservation(Base):  
    __tablename__ = 'reservations'  
    id = Column(Integer, primary_key=True, autoincrement=True)  
    customer_id = Column(String, ForeignKey('customers.id'))  
    hotel_id = Column(String)  
    room_type = Column(String)  
    check_in_date = Column(DateTime)  
    check_out_date = Column(DateTime)  
    status = Column(String)  # e.g., "booked", "cancelled", "checked_in", "checked_out"  
    # Add any other relevant fields that describe the room or the reservation


In [ ]:
engine = create_engine('sqlite:///data/hotel.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()
# Create a new instance of the Customer class
customer = Customer(id="12345", name="John Doe")

# Add the customer to the session
session.add(customer)

# Commit the changes to the database
session.commit()

print("Initial record has been inserted for customer_id 12345.")
# Create a new instance of the Customer class


In [ ]:
# Create a new instance of the Reservation class  
reservation = Reservation(  
    customer_id="12345",  
    hotel_id="H100",  
    room_type="Deluxe",  
    check_in_date=datetime(2023, 5, 20),  # Using a sample date, format is YYYY, MM, DD  
    check_out_date=datetime(2023, 5, 25), # Using a sample date, format is YYYY, MM, DD  
    status="booked"  
)  
  
# Add the reservation to the session  
session.add(reservation)  
  
# Commit the changes to the database for Reservation  
session.commit()  
  
print("Initial record has been inserted for customer_id 12345 with a reservation.")  

In [ ]:
def load_user_reservation_info(user_id):
    user_id = user_id.strip()
    matched_reservations = session.query(Reservation).filter_by(customer_id=user_id, status="booked").all()
    
    reservations_info = []
    for reservation in matched_reservations:
        reservation_info = {
            'room_type': reservation.room_type,
            'check_in_date': reservation.check_in_date.strftime('%Y-%m-%d'),
            'check_out_date': reservation.check_out_date.strftime('%Y-%m-%d'),
            'reservation_id': reservation.id,
            'hotel_id': reservation.hotel_id,
            'status': reservation.status
        }
        reservations_info.append(reservation_info)

    if not reservations_info:
        return "Sorry, we cannot find any reservation information for you."

    return str(reservations_info)
load_user_reservation_info("12345")